<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/Deep_Learning_Time_Series_Forecasting_using_Gluon_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install --upgrade mxnet-cu101==1.6.0
!pip install --upgrade mxnet==1.6.0
!pip install gluonts

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer

mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParams['axes.grid'] = False

In [ ]:
df= pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv")
df.head()

In [ ]:
color_list =[
             "blue",
             "orange",
             "green",
             "red",
             "purple",
             "brown",
             "pink",
             "gray",
             "olive",
             "cyan",
]

date_key="date"

In [ ]:
def visulalize(data):
  time_data=data[date_key]
  features = list(df.select_dtypes(include=[np.number]).columns
                  .values)
  feature_size =len(features)
  fig,axes = plt.subplots(
      nrows = int(np.ceil(feature_size/2)), ncols=2, figsize=(14,feature_size*2), dpi=80,facecolor="w",edgecolor="k"
  )

  for i in range(feature_size):
    key = features[i]
    c= color_list[i % (len(color_list))]
    t_data= data[key]
    t_data.index= time_data
    t_data.head()
    ax= t_data.plot(
        ax =axes[i //2, i%2],
        color= c,
        title = "{}".format(key),
        rot =25,
    ) 
    ax.legend([key])
    plt.tight_layout()
visulalize(df)

In [ ]:
df_input =df[['date','Appliances','T_out','Press_mm_hg','RH_out', 'Windspeed','Tdewpoint','Visibility']]

In [ ]:
df_input= df_input.set_index('date')

In [ ]:
train_time ="2016-05-10 00:00:00"
prediction_length= 144

In [ ]:
estimator = DeepAREstimator(freq="10min",
                            context_length=720,
                            prediction_length =prediction_length,
                            num_layers=2,
                            num_cells=128,
                            cell_type='lstm',
                            trainer =Trainer(epochs=8)
                            )

In [ ]:
from gluonts.dataset.common import ListDataset

training_data = ListDataset(
    [{"start":df_input.index[0],"target":df_input.Appliances[:train_time]}],
    freq="10min"
)

In [ ]:
predictor= estimator.train(training_data=training_data)

In [ ]:
test_data = ListDataset(
    [
     {"start":df_input.index[0],"target":df_input.Appliances[:"2016-05-11 00:00:00"]},
     {"start":df_input.index[0],"target":df_input.Appliances[:"2016-05-15 00:00:00"]}
    ],
    freq="10min"
)

In [ ]:
from gluonts.evaluation.backtest import make_evaluation_predictions

forecast_it,ts_it = make_evaluation_predictions(
    dataset =test_data,
    predictor= predictor,
    num_samples=100,
)

In [ ]:
forecasts= list(forecast_it)
tss= list(ts_it)

In [ ]:
forecasts

In [ ]:
forecast_entry= forecasts[0]

In [ ]:
forecast_entry

In [ ]:
def plot_prob_forecasts(ts_entry,forecast_entry):
  plot_length= prediction_length
  prediction_intervals=(80.0,95.0)
  legend= ["observations","median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

  fig,ax = plt.subplots(1,1, figsize=(10,7))
  ts_entry[-plot_length:].plot(ax=ax)
  forecast_entry.plot(prediction_intervals=prediction_intervals,color='g')
  plt.grid(which="both")
  plt.legend(legend,loc="upper left")
  plt.show()


In [77]:
forecasts[0]

gluonts.model.forecast.SampleForecast(freq="10min", info=None, item_id=None, samples=numpy.array([[58.49213790893555, 33.360008239746094, 36.81032180786133, 34.78863525390625, 30.428064346313477, 49.19782638549805, 58.83723831176758, 65.96251678466797, 57.670372009277344, 60.897499084472656, 68.06462097167969, 59.83902359008789, 59.01030731201172, 63.570472717285156, 56.403873443603516, 58.60500717163086, 45.44374084472656, 52.676170349121094, 37.37907791137695, 44.6497802734375, 50.64498519897461, 51.760894775390625, 62.564212799072266, 64.16666412353516, 46.44584655761719, 59.8408203125, 55.78657531738281, 55.418819427490234, 40.700103759765625, 59.381099700927734, 47.22171401977539, 50.74225616455078, 59.76688003540039, 71.73831939697266, 55.308414459228516, 49.539424896240234, 56.66236114501953, 55.69424819946289, 88.49159240722656, 75.98941802978516, 86.99786376953125, 49.748592376708984, 56.41069793701172, 61.6210823059082, 60.491737365722656, 65.91759490966797, 79.47660827636719

In [ ]:
plot_prob_forecasts(tss[0],forecasts[0])
plot_prob_forecasts(tss[1],forecasts[1])

In [ ]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.1,0.5,0.9])
agg_metrics,item_metrics=evaluator(iter(tss),iter(forecasts),num_series=len(test_data))

In [81]:
item_metrics

,item_id,MSE,abs_error,abs_target_sum,abs_target_mean,seasonal_error,MASE,MAPE,sMAPE,OWA,MSIS,QuantileLoss[0.1],Coverage[0.1],QuantileLoss[0.5],Coverage[0.5],QuantileLoss[0.9],Coverage[0.9]
0,NaN,673.363878,1677.581909,9510.0,66.041667,29.977297,0.388623,0.140598,0.151581,NaN,5.598819,927.961512,0.0,1677.581928,0.458333,1316.758595,0.965278
1,NaN,5765.994358,4404.809570,14350.0,99.652778,29.797792,1.026551,0.192115,0.235237,NaN,17.853919,1822.229617,0.0,4404.809265,0.201389,4420.516486,0.916667
